In [24]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
rating_data = pd.read_csv("C:/Users/Fafcom/Downloads/da/animelist.csv",nrows=50000000)
anima_data = pd.read_csv("C:/Users/Fafcom/Downloads/da/anime.csv")

# To save my Pc time I made a new data where i just fetch anime_id(MAL_ID) and Name so that i can use merge() function on it.
anima_data = anima_data.rename(columns={"MAL_ID": "anime_id"})
anima_contact_data = anima_data[["anime_id", "Name"]]

Preprocessing and applying all the methods for Recommendation System

In [35]:
rating_data = rating_data.merge(anima_contact_data, left_on = 'anime_id', right_on = 'anime_id', how = 'left')
rating_data = rating_data[["user_id", "Name", "anime_id","rating", "watching_status", "watched_episodes"]]
rating_data.head()

,user_id,Name,anime_id,rating,watching_status,watched_episodes
0,0,Basilisk: Kouga Ninpou Chou,67,9,1,1
1,0,Fairy Tail,6702,7,1,4
2,0,Gokusen,242,10,1,4
3,0,Kuroshitsuji,4898,0,1,1
4,0,One Piece,21,10,1,0


Now I will take only that data in which a particular anime has more than 200Votes and if a user has gave in total more than 500Votes to the anime.

In [42]:
count = rating_data['user_id'].value_counts()
count1 = rating_data['anime_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(count[count >= 500].index)].copy()
rating_data = rating_data[rating_data['anime_id'].isin(count1[count1 >= 200].index)].copy()

In [44]:
combine_movie_rating = rating_data.dropna(axis = 0, subset = ['Name'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['Name'])['rating'].
     count().
     reset_index()
     [['Name', 'rating']]
    )
movie_ratingCount.head()

,Name,rating
0,"""0""",553
1,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",2687
2,"""Bungaku Shoujo"" Memoire",3312
3,"""Bungaku Shoujo"" Movie",6087
4,"""Eiji""",298


We will combine movie_ratingCount and combine_movie_rating this will give us the most Known Anime name and filter out the lesser one

In [46]:
rating_data = combine_movie_rating.merge(movie_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_data = rating_data.drop(columns = "rating_x")
rating_data = rating_data.rename(columns={"rating_y": "rating"})
rating_data

,user_id,Name,anime_id,watching_status,watched_episodes,rating
0,6,Angel Beats! Specials,9062,1,1,12469
1,6,Ao no Exorcist,9919,1,2,24296
2,6,Blood+,150,1,15,12899
3,6,Casshern Sins,4981,1,12,8343
4,6,Guilty Crown,10793,1,2,23741
...,...,...,...,...,...,...
27117914,162163,Zetman,11837,2,13,7148
27117915,162163,Zettai Bouei Leviathan,17681,2,13,2729
27117916,162163,Zipang,29,2,26,1551
27117917,162163,_Summer,1692,2,2,1250


Encode the data

In [52]:
# Encoding categorical data
user_ids = rating_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_data["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_data["anime"] = rating_data["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min total rating: {}, Max total rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

Num of users: 30555, Num of animes: 9864
Min total rating: 136, Max total rating: 27651


In [54]:
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rating_data.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='anime_id')

pivot = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)

C:\Users\Fafcom\AppData\Local\Temp\ipykernel_12180\1509702243.py:9: FutureWarning: The provided callable <function sum at 0x0000020F3B0E5B20> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)


In [56]:
pivot.fillna(0, inplace=True)
pivot

anime_id,1535,1575,2001,2167,4224,5081,5114,6547,6746,9253,9989,10620,11111,11757,14813,15809,16498,19815,20507,30276
user_id,,,,,,,,,,,,,,,,,,,,
11100,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
20807,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
22022,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
34501,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
50485,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
61445,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
63900,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
68042,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507
82327,27110,26043,24552,24646,27298,26124,26062,27397,25271,27019,24821,25771,24347,27550,24380,25165,27651,26084,25458,25507


In [58]:
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table

user_id,6,12,17,19,21,42,44,47,53,60,...,162119,162124,162135,162136,162138,162141,162148,162157,162160,162163
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2687.0,0.0,...,0.0,2687.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3312.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6087.0,6087.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Eiji""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,4666.0,0.0,0.0,0.0,0.0,0.0,4666.0,0.0,...,0.0,4666.0,0.0,0.0,0.0,4666.0,0.0,0.0,0.0,0.0
xxxHOLiC Rou,0.0,0.0,4571.0,0.0,0.0,0.0,0.0,0.0,4571.0,0.0,...,0.0,4571.0,0.0,0.0,0.0,4571.0,0.0,0.0,0.0,0.0
xxxHOLiC Shunmuki,0.0,0.0,4977.0,0.0,0.0,0.0,0.0,0.0,4977.0,0.0,...,0.0,4977.0,0.0,0.0,0.0,4977.0,0.0,0.0,0.0,0.0


In [60]:
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [62]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [72]:
def predict():
    random_anime = np.random.choice(piviot_table.shape[0]) # This will choose a random anime name and our model will predict on it.

    query = piviot_table.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))

In [74]:
predict()

Recommendations for Double Decker! Doug & Kirill: Extra:

1: Double Decker! Doug & Kirill, with distance of 0.5306900014581761:
2: Mayonaka no Occult Koumuin: Fukurokouji to Ano Ko to Ore to, with distance of 0.7313578316595393:
3: Keishichou Tokumubu Tokushu Kyouakuhan Taisakushitsu Dainanaka: Tokunana OVA, with distance of 0.7354433222592902:
4: Golden Kamuy 2nd Season OVA, with distance of 0.7361182801917185:
5: Mayonaka no Occult Koumuin: Hitoribocchi no Kyuuketsuki, with distance of 0.7370866222248529:
